In [ ]:
#default_exp kernels

In [ ]:
#export 
import numba.cuda as cuda
import cmath as cm

@cuda.jit
def smatrix_forward_kernel(S, phase_factors, rho, r_min, out):
    """
    :param S:               B x NY x NX
    :param phase_factors:   B x D x K x 2
    :param rho:               D x K x 2
    :param out:             D x K x MY x MX
    :return: exit waves in out
    """
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    D, K, MY, MX, _ = out.shape
    B = S.shape[0]
    MM = MY * MX

    b = int(n // MM)
    my = (n - b * MM) // MX
    mx = (n - b * MM - my * MX)

    if n < B * MY * MX:
        for d in range(D):
            for k in range(K):
                # indexing with pixel precision
                rho0 = int(rho[d, k, 0] - r_min[0])
                rho1 = int(rho[d, k, 1] - r_min[1])

                a = S[b, rho0 + my, rho1 + mx, 0]
                c = S[b, rho0 + my, rho1 + mx, 1]
                u = phase_factors[b, d, k, 0]
                v = phase_factors[b, d, k, 1]

                val_real = a * u - c * v
                val_imag = c * u + a * v

                cuda.atomic.add(out, (d, k, my, mx, 0), val_real)
                cuda.atomic.add(out, (d, k, my, mx, 1), val_imag)

In [ ]:
#export
@cuda.jit
def smatrix_backward_kernel_S(z, phase_factors, mean_probe_intensities, r, r_min, out, tau):
    """
    S-matrix has beam tilts included, pre-calculated scanning phase factors.
    Fastest to compute

    :param z:                       D x K x MY x MX x 2
    :param phase_factors:           B x D x K x 2
    :param r:                       D x K x 2
    :param mean_probe_intensities:  D
    :param out:                     B x NY x NX x 2
    :param z_strides:  (4,)
    :return: exit waves in out
    """
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    D, K, MY, MX, _ = z.shape
    B = out.shape[0]

    b = n // (MY * MX)
    my = (n - b * (MX * MY)) // MX
    mx = (n - b * (MX * MY) - my * MX)

    if n < MY * MX * B:
        for d in range(D):
            for k in range(K):
                r0 = int(r[d, k, 0] - r_min[0])
                r1 = int(r[d, k, 1] - r_min[1])

                a = z[d, k, my, mx, 0]
                c = z[d, k, my, mx, 1]
                u = phase_factors[b, d, k, 0]
                v = phase_factors[b, d, k, 1]

                val_real = a * u + c * v
                val_imag = c * u - a * v

                val_real *= tau[0] / mean_probe_intensities[d]
                val_imag *= tau[0] / mean_probe_intensities[d]

                cuda.atomic.add(out, (b, r0 + my, r1 + mx, 0), val_real)
                cuda.atomic.add(out, (b, r0 + my, r1 + mx, 1), val_imag)

In [ ]:
#export
@cuda.jit
def phase_factor_kernelDBK(Psi, rho, qB, out):
    """
    Calculate the phase factors (due to beam scan) probe wave function so that
    the probe is scanned to the correct place for each diffraction pattern

    :param Psi:         D x B
        Probe wave functions Fourier coefficient for each beam to be mutliplied
        by phase factor to account for beam scan position
    :param rho:           D x K x 2
        Probe positions in pixels
    :param qB:          2 x B
        Fourier space coordinates of the beams
    :param out:         D x B x K x 2
        Phase factors output
    :param out_strides: (3,)
    :return: scanning phases for all defoc, beams, positions
    """
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    B, D, K, c = out.shape
    N = B * K * D

    b = n // (K * D)
    d = (n - b * (K * D)) // K
    k = (n - b * (K * D) - d * K)

    if n < N:
        rho0 = rho[d, k, 0]
        rho1 = rho[d, k, 1]
        Psic = Psi[d, b, 0] + 1j * Psi[d, b, 1]
        # scanning phase with subpixel precision
        v = cm.exp(-2j * cm.pi * (qB[0, b] * rho0 + qB[1, b] * rho1)) * Psic
        out[b, d, k, 0] = v.real
        out[b, d, k, 1] = v.imag

In [ ]:
#export
@cuda.jit
def phase_factor_kernelKB(Psi, rho, qB, out):
    """
    Calculate the phase factors (due to beam scan) probe wave function so that
    the probe is scanned to the correct place for each diffraction pattern

    :param Psi:         B x 2
        Probe wave functions Fourier coefficient for each beam to be mutliplied
        by phase factor to account for beam scan position
    :param rho:         K x 2
        Probe positions in pixels
    :param qB:          2 x B
        Fourier space coordinates of the beams
    :param out:         K x B x 2
        Phase factors output
    :return: scanning phases for all defoc, beams, positions
    """
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    K, B, c = out.shape
    N = B * K

    b = n // (K)
    k = (n - b * K)

    if n < N:
        rho0 = rho[k, 0]
        rho1 = rho[k, 1]
        Psic = Psi[b, 0] + 1j * Psi[b, 1]
        # scanning phase with subpixel precision
        v = cm.exp(-2j * cm.pi * (qB[0, b] * rho0 + qB[1, b] * rho1)) * Psic
        out[k, b, 0] = v.real
        out[k, b, 1] = v.imag

In [ ]:
#export